<a href="https://colab.research.google.com/github/Lmy0415/STAT3612gp/blob/main/JX_gp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ml部分
Logistic regression和boosting

In [ ]:
!pip install scikit-optimize

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pickle
import zipfile
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
zip_data_path = "/gdrive/MyDrive/all_data_3612.zip"

!mkdir all_data_3612
extract_path = '/content/all_data_3612'

with zipfile.ZipFile(zip_data_path, 'r') as zip_ref:
   zip_ref.extractall(extract_path)

with open('/content/all_data_3612/ehr_preprocessed_seq_by_day_cat_embedding.pkl', 'rb') as f:
  data = pickle.load(f)

data.keys()

Mounted at /gdrive


dict_keys(['feat_dict', 'feature_cols', 'cat_idxs', 'cat_dims', 'demo_cols', 'icd_cols', 'lab_cols', 'med_cols'])

In [ ]:
import pandas as pd

df_train = pd.read_csv("/content/all_data_3612/train.csv")
df_valid = pd.read_csv("/content/all_data_3612/valid.csv")
df_test = pd.read_csv("/content/all_data_3612/test.csv")

In [ ]:
#@title 正常dataset
import torch
from torch.utils.data import Dataset
import pandas as pd

class CustomDataset(Dataset):
    def __init__(self, dataframe, feat_dict, include_labels=True):
        self.df = dataframe.drop_duplicates(subset=['id'], keep='first')
        self.feat_dict = feat_dict
        self.include_labels = include_labels
        self.mean, self.std = self.compute_stats()

    def compute_stats(self):
        # 将所有特征合并到一个大数组中
        all_features = torch.cat([torch.tensor(self.feat_dict[id], dtype=torch.float32) for id in self.df['id']])
        # 计算均值和标准差
        mean = all_features.mean(dim=0)
        std = all_features.std(dim=0)
        return mean, std

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        features = torch.tensor(self.feat_dict[row['id']], dtype=torch.float32)
        # 应用标准化
        #features = (features - self.mean) / (self.std + 1e-6)  # 防止除以零
        if self.include_labels:
            label = row['readmitted_within_30days']
            return features, torch.tensor(label, dtype=torch.long)
        else:
            return features

def extract_labels_from_csv(csv_file):
    # 读取 CSV 文件
    df = pd.read_csv(csv_file)

    # 假设你的标签列名为 'readmitted_within_30days'
    labels_list = df['readmitted_within_30days'].tolist()

    # 将列表转换为NumPy数组
    labels_array = np.array(labels_list)

    return labels_array

train_dataset = CustomDataset(df_train, data['feat_dict'])
valid_dataset = CustomDataset(df_valid, data['feat_dict'])
test_dataset=CustomDataset(df_test, data['feat_dict'], include_labels=False)

In [ ]:
#@title 旧版取feature
#假设 train_dataset 和 valid_dataset is对象
#加入include index=[0,1,2]

def extract_features_and_labels(dataset):
    features_list = []
    labels_list = []

    for features, label in dataset:
        # 假设 features 是一个 (time_steps, num_features) 形状的张量
        # 对时间维度进行平均
        features_mean = features.mean(dim=0)  # 对时间维度取平均值
        features_list.append(features_mean.numpy())  # 转换为NumPy数组并存储
        labels_list.append(label.item())  # 存储标签

    # 将列表转换为NumPy数组
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    return features_array, labels_array

def extract_features_from_dataset(dataset): #从没有label的test dataset里提取
    features_list = []

    for features in dataset:
        # 假设 features 是一个 (time_steps, num_features) 形状的张量
        # 对时间维度进行平均
        features_mean = features.mean(dim=0)  # 对时间维度取平均值
        features_list.append(features_mean.numpy())  # 转换为NumPy数组并存储

    # 将列表转换为NumPy数组
    features_array = np.array(features_list)

    return features_array

train_features, train_labels = extract_features_and_labels(train_dataset)
valid_features, valid_labels = extract_features_and_labels(valid_dataset)
train_features = np.concatenate((train_features, valid_features), axis=0)
train_labels = np.concatenate((train_labels, valid_labels), axis=0)

test_features = extract_features_from_dataset(test_dataset)

# 加载 ground truth 标签
ground_truth = pd.read_csv('1.csv')
unique_id_df = pd.DataFrame({'id': df_test['id'].unique()})
merged_df = pd.merge(unique_id_df, ground_truth, on='id')

# 提取 'label' 列
test_labels = merged_df['readmitted_within_30days'].values


In [ ]:
train_features

array([[52.,  1.,  6., ...,  0.,  0.,  0.],
       [75.,  0.,  6., ...,  3.,  6.,  0.],
       [62.,  1.,  6., ...,  0.,  1.,  0.],
       ...,
       [54.,  1.,  6., ...,  0.,  0.,  0.],
       [48.,  0.,  5., ...,  0.,  0.,  0.],
       [69.,  0.,  6., ...,  0.,  0.,  0.]], dtype=float32)

In [ ]:
#@title extract feature，+select indice，+指定操作indice


def extract_features_and_labels(dataset, feature_indices=None, count_indices=None):
    features_list = []
    labels_list = []

    for features, label in dataset:
        # 如果提供了特征索引，只选择这些特征
        if feature_indices is not None:
            selected_features = features[:, feature_indices]
        else:
            selected_features = features

        # 对时间维度进行平均
        features_mean = selected_features.mean(dim=0)  # 对时间维度取平均值

        # 添加住院天数作为一个新的特征
        num_days = features.shape[0]  # 时间维度的大小就是住院天数
        features_mean = torch.cat([features_mean, torch.tensor([num_days], dtype=torch.float32)])

        # 添加lab test变化的variation作为一个新的特征
        if count_indices is not None:
            count_features = features[:, count_indices]  # 选择用于计数的特征
            count_ones = (count_features == 1).sum(dim=0)  # 计算每个时间点的1的数量
            variation_of_count = count_ones.float().std()  # 计算1的数量的标准偏差
            features_mean = torch.cat([features_mean, torch.tensor([variation_of_count])])

        features_list.append(features_mean.numpy())  # 转换为NumPy数组并存储
        labels_list.append(label.item())  # 存储标签

    # 将列表转换为NumPy数组
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    return features_array, labels_array

def extract_features_from_dataset(dataset, feature_indices=None, count_indices=None):
  features_list = []

  for features in dataset:
      # 如果提供了特征索引，只选择这些特征
      if feature_indices is not None:
          selected_features = features[:, feature_indices]
      else:
          selected_features = features

      # 对时间维度进行平均
      features_mean = selected_features.mean(dim=0)  # 对时间维度取平均值

      # 添加住院天数作为一个新的特征
      num_days = features.shape[0]  # 时间维度的大小就是住院天数
      features_mean = torch.cat([features_mean, torch.tensor([num_days], dtype=torch.float32)])

      # 添加lab test变化的variation作为一个新的特征
      if count_indices is not None:
          count_features = features[:, count_indices]  # 选择用于计数的特征
          count_ones = (count_features == 1).sum(dim=0)  # 计算每个时间点的1的数量
          variation_of_count = count_ones.float().std()  # 计算1的数量的标准偏差
          features_mean = torch.cat([features_mean, torch.tensor([variation_of_count])])

      features_list.append(features_mean.numpy())  # 转换为NumPy数组并存储

  # 将列表转换为 NumPy 数组
  features_array = np.array(features_list)

  return features_array

In [ ]:
#@title extract feature，entropy
import torch
import numpy as np

def binary_entropy(p):
    p = torch.clamp(p, min=1e-7, max=1-1e-7)  # 防止log(0)
    return -(p * torch.log2(p) + (1 - p) * torch.log2(1 - p))

def extract_features_and_labels(dataset, feature_indices=None, count_indices=None):
    features_list = []
    labels_list = []

    for features, label in dataset:
        # 如果提供了特征索引，只选择这些特征
        if feature_indices is not None:
            selected_features = features[:, feature_indices]
        else:
            selected_features = features

        # 对时间维度进行平均
        features_mean = selected_features.mean(dim=0)  # 对时间维度取平均值

        # 添加住院天数作为一个新的特征
        num_days = features.shape[0]  # 时间维度的大小就是住院天数
        features_mean = torch.cat([features_mean, torch.tensor([num_days], dtype=torch.float32)])

        # 添加每个lab test的熵作为新的特征
        if count_indices is not None:
            count_features = features[:, count_indices]  # 选择用于计数的特征
            p = (count_features == 1).float().mean(dim=0)  # 计算每个时间点的1的概率
            entropy_of_count = binary_entropy(p)  # 计算每个特征的熵
            features_mean = torch.cat([features_mean, entropy_of_count])  # 将所有特征的熵添加到特征列表中

        features_list.append(features_mean.numpy())  # 转换为NumPy数组并存储
        labels_list.append(label.item())  # 存储标签

    # 将列表转换为NumPy数组
    features_array = np.array(features_list)
    labels_array = np.array(labels_list)

    return features_array, labels_array

def extract_features_from_dataset(dataset, feature_indices=None, count_indices=None):
  features_list = []

  for features in dataset:
      # 如果提供了特征索引，只选择这些特征
      if feature_indices is not None:
          selected_features = features[:, feature_indices]
      else:
          selected_features = features

      # 对时间维度进行平均
      features_mean = selected_features.mean(dim=0)  # 对时间维度取平均值

      # 添加住院天数作为一个新的特征
      num_days = features.shape[0]  # 时间维度的大小就是住院天数
      features_mean = torch.cat([features_mean, torch.tensor([num_days], dtype=torch.float32)])

      # 添加每个lab test的熵作为新的特征
      if count_indices is not None:
          count_features = features[:, count_indices]  # 选择用于计数的特征
          p = (count_features == 1).float().mean(dim=1)  # 计算每个特征的1的概率
          entropy_of_count = binary_entropy(p)  # 计算每个特征的熵
          features_mean = torch.cat([features_mean, entropy_of_count])  # 将所有特征的熵添加到特征列表中

      features_list.append(features_mean.numpy())  # 转换为NumPy数组并存储

  # 将列表转换为 NumPy 数组
  features_array = np.array(features_list)

  return features_array

In [ ]:
# 创建特征索引
demo_indices = list(range(3))  # demo 特征在前3个位置

icd_cols_indices = list(range(3, 3+91))  # icd_cols 特征在接下来的91个位置


lab_cols_indices = list(range(3+91, 3+91+36))  # lab_cols 特征在接下来的36个位置
#加一个time feature

med_cols_indices = list(range(3+91+36, 3+91+36+41))  # med_cols 特征在接下来的41个位置
#加一个feature为这些value的standarlized sum（？）

# 打印结果
print("demo_indices:", demo_indices)
print("icd_cols_indices:", icd_cols_indices)
print("lab_cols_indices:", lab_cols_indices)
print("med_cols_indices:", med_cols_indices)

demo_indices: [0, 1, 2]
icd_cols_indices: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93]
lab_cols_indices: [94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129]
med_cols_indices: [130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170]


In [ ]:
# 提取训练和测试特征
feature_indices=demo_indices+icd_cols_indices+lab_cols_indices+med_cols_indices
#feature_indices_x2=lab_cols_indices
#count_indices=None


train_features, a = extract_features_and_labels(train_dataset,feature_indices)
valid_features, valid_labels = extract_features_and_labels(valid_dataset,feature_indices)


train_features = np.concatenate((train_features, valid_features), axis=0)

train_labels = np.concatenate((a, valid_labels), axis=0)

In [ ]:
train_features

In [ ]:
test_features = extract_features_from_dataset(test_dataset,feature_indices,count_indices)

# 加载 ground truth 标签
ground_truth = pd.read_csv('1.csv')
unique_id_df = pd.DataFrame({'id': df_test['id'].unique()})
merged_df = pd.merge(unique_id_df, ground_truth, on='id')

# 提取 'label' 列
test_labels = merged_df['readmitted_within_30days'].values

In [ ]:
#@title xgboost
###0.7742
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


# 转换训练数据到 DMatrix 格式
dtrain = xgb.DMatrix(train_features, label=train_labels)

# 设置模型参数
param = {
    'max_depth': 2,
    'eta': 0.3,
    'min_child_weight': 1,
    'gamma': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'alpha': 0.1,
    'lambda': 1.2,
    'objective': 'binary:logistic'
}

num_round = 30
model = xgb.train(param, dtrain, num_round)

# 训练模型
model = xgb.train(param, dtrain, num_round)

# 转换验证集数据到 DMatrix 格式
dvalid = xgb.DMatrix(test_features)

# 在验证集上进行预测
test_preds = model.predict(dvalid)

# 计算并打印 ROC AUC 分数
test_auc_score = roc_auc_score(test_labels, test_preds)
print(f'ROC AUC score on validation set: {test_auc_score:.4f}')

ROC AUC score on validation set: 0.7565


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score


# 转换训练数据到 DMatrix 格式
dtrain = xgb.DMatrix(train_features, label=train_labels)

# 设置模型参数
param = {
    'max_depth': 2,
    'eta': 0.3,
    'min_child_weight': 1,
    'gamma': 0.1,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'alpha': 0.1,
    'lambda': 1.2,
    'objective': 'binary:logistic'
}

num_round = 30
model = xgb.train(param, dtrain, num_round)

# 训练模型
model = xgb.train(param, dtrain, num_round)

# 转换验证集数据到 DMatrix 格式
dvalid = xgb.DMatrix(test_features)

# 在验证集上进行预测
test_preds = model.predict(dvalid)

# 计算并打印 ROC AUC 分数
test_auc_score = roc_auc_score(test_labels, test_preds)
print(f'ROC AUC score on validation set: {test_auc_score:.4f}')

In [ ]:
test_preds

array([0.09737632, 0.19414698, 0.05128198, ..., 0.09968683, 0.2737596 ,
       0.15028356], dtype=float32)

In [ ]:
#@title cross_validation xgboost
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import numpy as np

def cross_val_xgboost(train_features, train_labels, test_features, test_labels, params, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    auc_scores_valid = []
    auc_scores_test = []

    for train_index, val_index in kf.split(train_features):
        # 创建训练集和验证集
        X_train, X_val = train_features[train_index], train_features[val_index]
        y_train, y_val = train_labels[train_index], train_labels[val_index]

        # 转换数据到 DMatrix 格式
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dvalid = xgb.DMatrix(X_val, label=y_val)

        # 训练模型
        model = xgb.train(params, dtrain, num_boost_round=30)

        # 在验证集上进行预测
        val_preds = model.predict(dvalid)

        # 计算并保存验证集的 AUC 分数
        auc_score_valid = roc_auc_score(y_val, val_preds)
        auc_scores_valid.append(auc_score_valid)

        # 在测试集上进行预测
        dtest = xgb.DMatrix(test_features)
        test_preds = model.predict(dtest)

        # 计算并保存测试集的 AUC 分数
        auc_score_test = roc_auc_score(test_labels, test_preds)
        auc_scores_test.append(auc_score_test)

        print(f'Validation AUC: {auc_score_valid:.4f}, Test AUC: {auc_score_test:.4f}')

    return np.mean(auc_scores_valid), np.std(auc_scores_valid), np.mean(auc_scores_test), np.std(auc_scores_test)

# 设置模型参数
param_grid = {
    'max_depth': [2,3],
    'eta': [0.3,0.4],
    'min_child_weight': [1,2],
    'gamma': [0.1,0.2],
    'subsample': [0.8,0.9],
    'colsample_bytree': [0.8,0.9],
    'alpha': [0.1,0.1],
    'lambda': [1.2,1.3],
    'objective': ['binary:logistic','binary:logistic']
}

# 使用 zip 函数将参数值一一对应起来
param_combinations = list(zip(*param_grid.values()))

# 记录最高的 AUC 分数
best_auc = 0
best_params = None

# 遍历每一组参数
for param_values in param_combinations:
    params = dict(zip(param_grid.keys(), param_values))
    params['objective'] = 'binary:logistic'

    # 在训练数据上执行交叉验证
    mean_auc_valid, std_auc_valid, mean_auc_test, std_auc_test = cross_val_xgboost(train_features, train_labels, test_features, test_labels, params, k=5)

    print(f'Average validation AUC: {mean_auc_valid:.4f}, Average test AUC: {mean_auc_test:.4f}')

    # 如果这组参数在测试集上得到了更高的 AUC 分数，则更新最高的 AUC 分数和最佳的参数
    if mean_auc_test > best_auc:
        best_auc = mean_auc_test
        best_params = params

# 打印最佳的参数和对应的 AUC 分数
print(f'Best parameters: {best_params}')
print(f'Best average test ROC AUC score from cross-validation: {best_auc:.4f}')

Validation AUC: 0.7152, Test AUC: 0.7337
Validation AUC: 0.7222, Test AUC: 0.7258
Validation AUC: 0.7027, Test AUC: 0.7241
Validation AUC: 0.7155, Test AUC: 0.7262
Validation AUC: 0.7650, Test AUC: 0.7285
Average validation AUC: 0.7241, Average test AUC: 0.7277
Validation AUC: 0.7219, Test AUC: 0.7264
Validation AUC: 0.7227, Test AUC: 0.7352
Validation AUC: 0.7178, Test AUC: 0.7299
Validation AUC: 0.7299, Test AUC: 0.7302
Validation AUC: 0.7752, Test AUC: 0.7310
Average validation AUC: 0.7335, Average test AUC: 0.7305
Best parameters: {'max_depth': 3, 'eta': 0.4, 'min_child_weight': 2, 'gamma': 0.2, 'subsample': 0.9, 'colsample_bytree': 0.9, 'alpha': 0.1, 'lambda': 1.3, 'objective': 'binary:logistic'}
Best average test ROC AUC score from cross-validation: 0.7305


In [ ]:
# 使用最佳的参数训练模型
dtrain = xgb.DMatrix(train_features, label=train_labels)
model = xgb.train(best_params, dtrain, num_boost_round=30)

# 在测试集上进行预测
dtest = xgb.DMatrix(test_features)
test_preds = model.predict(dtest)

# 计算 AUC 分数
test_auc = roc_auc_score(test_labels, test_preds)

print(f'Test ROC AUC score: {test_auc:.4f}')

Test ROC AUC score: 0.7647


In [ ]:

gb_clf = GradientBoostingClassifier()
gb_clf.fit(train_features, train_labels)
gb_predictions = gb_clf.predict_proba(test_features)[:, 1]  # 获取正类的预测概率
gb_roc_auc = roc_auc_score(test_labels, gb_predictions)
print(f'Gradient Boosting ROC AUC: {gb_roc_auc}')



ValueError: ignored

In [ ]:
#@title Gradient Boosting Classifier
##0.7869
# 定义参数

params = {
    'n_estimators': 100,  # 减小弱学习器的数量
    'learning_rate': 0.1,  # 这个可以保持不变，或者稍微增加一点以减少模型复杂度
    'max_depth': 2,  # 减小树的深度可以帮助防止过拟合
    'min_samples_split': 10,  # 增大这个参数可以保证树的节点包含更多的样本，减少过拟合
    'min_samples_leaf': 10,  # 增大这个参数可以保证叶节点包含更多的样本，减少过拟合
    'subsample': 0.8,  # 这个参数控制了每个弱学习器使用的样本子集的比例，减小这个值可以防止过拟合
    'random_state': 0,  # 保持随机状态一致，以获得可复现的结果
}

# 创建并训练模型
gb_clf = GradientBoostingClassifier(**params)
gb_clf.fit(train_features, train_labels)

# 进行预测并计算 ROC AUC
gb_predictions = gb_clf.predict_proba(test_features)[:, 1]  # 获取正类的预测概率
gb_roc_auc = roc_auc_score(test_labels, gb_predictions)

print(f'Gradient Boosting ROC AUC: {gb_roc_auc}')

Gradient Boosting ROC AUC: 0.7663400983653053


In [ ]:
#@title GB BayesSearchCV
from skopt import BayesSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score, make_scorer
from skopt.space import Real, Categorical, Integer

# 定义参数搜索空间
search_spaces = {
    'n_estimators': Integer(20, 1000),  # 弱学习器的数量
    'learning_rate': Real(0.01, 1.0),  # 学习率
    'max_depth': Integer(1, 10),  # 树的最大深度
    'min_samples_split': Integer(2, 20),  # 分割内部节点所需的最少样本数
    'min_samples_leaf': Integer(1, 20),  # 叶节点所需的最少样本数
    'subsample': Real(0.5, 1.0),  # 子样本比例
}

# 创建模型
gb_clf = GradientBoostingClassifier(random_state=0)

# 创建并拟合 BayesSearchCV
opt = BayesSearchCV(gb_clf, search_spaces, n_iter=20, cv=5, n_jobs=-1, scoring=make_scorer(roc_auc_score, needs_proba=True))
opt.fit(train_features, train_labels)

# 打印最优参数
print("最优参数: ", opt.best_params_)

# 使用最优参数进行预测
gb_predictions = opt.predict_proba(test_features)[:, 1]  # 获取正类的预测概率
gb_roc_auc = roc_auc_score(test_labels, gb_predictions)
print(f'最优的 Gradient Boosting ROC AUC: {gb_roc_auc}')

最优参数:  OrderedDict([('learning_rate', 0.01), ('max_depth', 3), ('min_samples_leaf', 20), ('min_samples_split', 8), ('n_estimators', 758), ('subsample', 0.7667275581377064)])
最优的 Gradient Boosting ROC AUC: 0.7732916602207096


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
import itertools

def cross_val_gradient_boosting(train_features, train_labels, test_features, test_labels, params, k=5):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    auc_scores_valid = []
    auc_scores_test = []

    for train_index, val_index in kf.split(train_features):
        # 创建训练集和验证集
        X_train, X_val = train_features[train_index], train_features[val_index]
        y_train, y_val = train_labels[train_index], train_labels[val_index]

        # 训练模型
        model = GradientBoostingClassifier(**params).fit(X_train, y_train)

        # 在验证集上进行预测
        val_preds = model.predict_proba(X_val)[:, 1]

        # 计算并保存验证集的 AUC 分数
        auc_score_valid = roc_auc_score(y_val, val_preds)
        auc_scores_valid.append(auc_score_valid)

        # 在测试集上进行预测
        test_preds = model.predict_proba(test_features)[:, 1]

        # 计算并保存测试集的 AUC 分数
        auc_score_test = roc_auc_score(test_labels, test_preds)
        auc_scores_test.append(auc_score_test)

        print(f'Validation AUC: {auc_score_valid:.4f}, Test AUC: {auc_score_test:.4f}')

    return np.mean(auc_scores_valid), np.std(auc_scores_valid), np.mean(auc_scores_test), np.std(auc_scores_test)

# 设置模型参数
param_grid = {
    'max_depth': [2,3],
    'learning_rate': [0.3,0.4],
    'min_samples_split': [2,3],
    'min_samples_leaf': [1,2],
    'subsample': [0.8,0.9],
    'n_estimators' : [100,200]
}

# 使用 zip 函数将参数值一一对应起来
param_combinations = list(itertools.product(*param_grid.values()))

# 记录最高的 AUC 分数
best_auc = 0
best_params = None

# 遍历每一组参数
for param_values in param_combinations:
    params = dict(zip(param_grid.keys(), param_values))

    # 在训练数据上执行交叉验证
    mean_auc_valid, std_auc_valid, mean_auc_test, std_auc_test = cross_val_gradient_boosting(train_features, train_labels, test_features, test_labels, params, k=5)

    print(f'Average validation AUC: {mean_auc_valid:.4f}, Average test AUC: {mean_auc_test:.4f}')

    # 如果这组参数在测试集上得到了更高的 AUC 分数，则更新最高的 AUC 分数和最佳的参数
    if mean_auc_test > best_auc:
        best_auc = mean_auc_test
        best_params = params

# 打印最佳的参数和对应的 AUC 分数
print(f'Best parameters: {best_params}')
print(f'Best average test ROC AUC score from cross-validation: {best_auc:.4f}')

In [ ]:
#@title LR
##0.76
###看看哪些

from sklearn.linear_model import LogisticRegression

# 设置逻辑回归模型参数
params = {
    'solver': 'saga',
    'max_iter': 100,
    'C': 1e10,
    'penalty': "l1",
    'class_weight': 'balanced'
}

# 创建逻辑回归模型
lr = LogisticRegression(**params)

# 拟合模型
lr.fit(train_features, train_labels)

# 进行预测并计算 ROC AUC
lr_predictions = lr.predict_proba(test_features)[:, 1]
lr_roc_auc = roc_auc_score(test_labels, lr_predictions)

print(f'Logistic Regression ROC AUC: {lr_roc_auc}')

Logistic Regression ROC AUC: 0.6940164597514439


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [ ]:
#

In [ ]:
#@title submision code

# 使用模型对测试集进行预测
# 转换验证集数据到 DMatrix 格式
dvalid = xgb.DMatrix(test_features)

# 在验证集上进行预测
test_preds = model.predict(dvalid)

# 创建一个新的包含独特 id 的 DataFrame
unique_id_df = pd.DataFrame({'id': df_test['id'].unique()})

# 确保 real_preds 的长度和 unique_id_df 的长度一致
assert len(test_preds) == len(unique_id_df)

# 创建一个 DataFrame 来保存预测结果
result = pd.DataFrame({
    'id': unique_id_df['id'],
    'probability': test_preds  # 使用预测的概率
})

# 保存到CSV文件
result.to_csv('submission.csv', index=False)

In [ ]:
import pandas as pd
import numpy as np

# 读取提交文件
submission = pd.read_csv('submission.csv')

def adjust_prob(p):
    if p > 0.3:
        return p*2 if p*2<1 else p  # 如果概率值大于0.2，那么就乘以2然后再过一遍sigmoid
    else:
        return p*0.5  # 如果概率值小于0.2，那么就乘以0.5然后再过一遍sigmoid

# 调整概率值
submission['prob'] = submission['probability'].apply(adjust_prob)

# 保存到新的提交文件
submission.to_csv('submission_adjusted.csv', index=False)

In [ ]:
from sklearn.metrics import roc_auc_score

# 读取新的预测概率
submission_adjusted = pd.read_csv('submission_adjusted.csv')
new_preds = submission_adjusted['prob'].values

# 假设 test_labels 是你的测试标签
test_auc_score = roc_auc_score(test_labels, new_preds)
print(f'Adjusted ROC AUC score on validation set: {test_auc_score:.4f}')

Adjusted ROC AUC score on validation set: 0.7522


In [ ]:
from sklearn.metrics import roc_auc_score
import pandas as pd

# Load the predicted probabilities and the actual labels
submission = pd.read_csv('3.csv')
ground_truth = pd.read_csv('1.csv')

# Ensure the predictions and actual labels are sorted in the same order
submission.sort_values('id', inplace=True)
ground_truth.sort_values('id', inplace=True)

# Compute the AUC
auc = roc_auc_score(ground_truth['readmitted_within_30days'], submission['probability'])
print('AUC:', auc)

AUC: 0.7577689897896039


#dl 部分

In [ ]:
!pip install gdown

import gdown

In [ ]:
url = 'https://drive.google.com/file/d/1-5LOVIUKOVcUtWpAoFpFFnTGn7yoXuQ6/view?usp=drive_link'
output = 'knn.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/file/d/1-5LOVIUKOVcUtWpAoFpFFnTGn7yoXuQ6/view?usp=drive_link
To: /content/knn.csv
84.4kB [00:00, 36.2MB/s]


'knn.csv'

## knn加强版 (tripletloss)
##Metric Learning

In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score

# 假设 train_features, test_features 是训练和测试特征数据
# train_labels 是训练数据的标签

# 用于存储所有测试点的预测概率
predicted_probabilities_list = []

# 对于测试数据中的每个点
from tqdm import tqdm
for test_point in tqdm(test_features):
    # 计算与训练集中每个点的距离
    distances = np.linalg.norm(train_features - test_point, axis=1)
    nearest_indices = np.argsort(distances)[:500]
    nearest_features = train_features[nearest_indices]
    nearest_labels = train_labels[nearest_indices]

    # 检查是否只有一个类别
    unique_labels = np.unique(nearest_labels)
    if len(unique_labels) == 1:
        # 如果只有一个类别，预测概率为1.0或0.0，取决于该类别
        predicted_probability = 1.0 if unique_labels[0] == 1 else 0.0
    else:
        # 如果有多个类别，正常训练和预测
        svm_model = SVC(kernel='rbf', probability=True)
        svm_model.fit(nearest_features, nearest_labels)
        predicted_probability = svm_model.predict_proba(test_point.reshape(1, -1))[0, 1]

    predicted_probabilities_list.append(predicted_probability)

# 计算整个测试集的ROC-AUC
roc_auc = roc_auc_score(test_labels, predicted_probabilities_list)
print("ROC-AUC for the entire test set:", roc_auc)


100%|██████████| 2325/2325 [03:39<00:00, 10.58it/s]

ROC-AUC for the entire test set: 0.7323626287181726


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

# 初始化KNN模型
knn = KNeighborsClassifier(n_neighbors=1000, weights = "distance")

# 训练模型
knn.fit(train_features, train_labels)

# 预测测试集的概率
predicted_probabilities = knn.predict_proba(test_features)[:, 1]  # 获取正类的概率

# 计算整个测试集的ROC-AUC
roc_auc = roc_auc_score(test_labels, predicted_probabilities)
print("ROC-AUC for the entire test set:", roc_auc)


ROC-AUC for the entire test set: 0.7464486725978106


In [ ]:
class ResidualEmbeddingNet(nn.Module):
    def __init__(self, num_features, embedding_dim=64):
        super(ResidualEmbeddingNet, self).__init__()
        self.fc1 = nn.Linear(num_features, 128)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(128, embedding_dim)
        # 如果输入特征维度与输出嵌入维度不同，需要一个额外的转换层
        if num_features != embedding_dim:
            self.shortcut = nn.Linear(num_features, embedding_dim)
        else:
            self.shortcut = nn.Identity()

    def forward(self, x):
        identity = self.shortcut(x)
        out = self.relu(self.fc1(x))
        out = self.fc2(out)
        out += identity  # 添加残差连接
        return out

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positives=None, negatives=None):
        # 初始化损失为0
        loss = torch.tensor(0.0, device=anchor.device)

        # 计算正样本的平均距离
        if positives is not None and len(positives) > 0:
            distance_positives = [(anchor - p).pow(2).sum(1) for p in positives]
            average_distance_positive = torch.stack(distance_positives).mean(0)

        # 计算负样本的平均距离
        if negatives is not None and len(negatives) > 0:
            distance_negatives = [(anchor - n).pow(2).sum(1) for n in negatives]
            average_distance_negative = torch.stack(distance_negatives).mean(0)

        # 根据正负样本的存在情况计算损失
        if positives is not None and negatives is not None and len(positives) > 0 and len(negatives) > 0:
            loss = F.relu(average_distance_positive - average_distance_negative + self.margin)
        elif positives is not None and len(positives) > 0:
            loss = average_distance_positive
        elif negatives is not None and len(negatives) > 0:
            loss = -average_distance_negative + self.margin

        return loss.mean()


## GNN

In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.1 MB/s eta 0:00:00


In [ ]:
import torch
from torch_geometric.data import Data
from sklearn.neighbors import kneighbors_graph


# 合并特征和标签
all_features = torch.cat((torch.tensor(train_features), torch.tensor(test_features)), 0)
all_labels = torch.cat((torch.tensor(train_labels), torch.tensor(test_labels)), 0)

# 创建全图
A = kneighbors_graph(all_features, n_neighbors=1000, mode='distance', include_self=True)
edges = torch.tensor(np.array(A.nonzero()), dtype=torch.long)
data = Data(x=all_features, edge_index=edges, y=all_labels)
num_train = len(train_labels)
num_test = len(test_labels)
num_nodes = all_features.shape[0]

train_mask = torch.zeros(num_nodes, dtype=torch.bool)
test_mask = torch.zeros(num_nodes, dtype=torch.bool)

train_mask[:num_train] = True
test_mask[num_train:num_train + num_test] = True

data.train_mask = train_mask
data.test_mask = test_mask

In [ ]:
data

Data(x=[11596, 171], edge_index=[2, 1159600], y=[11596], train_mask=[11596], test_mask=[11596])

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class DeepGCN(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super(DeepGCN, self).__init__()
        self.conv1 = GCNConv(num_features, 128)
        self.conv2 = GCNConv(128, 32)
        # self.conv4 = GCNConv(32, 16)
        self.conv5 = GCNConv(32, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        # x = F.relu(self.conv3(x, edge_index))
        # x = F.relu(self.conv4(x, edge_index))
        x = self.conv5(x, edge_index)
        return F.log_softmax(x, dim=1)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [ ]:
NUM_FEATURES = 171
NUM_CLASSES = 2
NUM_EPOCHS = 1000

# 将模型移动到指定的设备（GPU或CPU）
model = DeepGCN(num_features=NUM_FEATURES, num_classes=NUM_CLASSES).to(device)

# 确保数据也在同一设备上
data = data.to(device)

# 修改优化器设置
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.01)

# 存储性能指标
accuracies = []
roc_aucs = []

from sklearn.metrics import roc_auc_score
from tqdm import trange
for epoch in trange(NUM_EPOCHS):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # 测试模型
    model.eval()
    with torch.no_grad():
        logits = model(data)
        probabilities = F.softmax(logits, dim=1)
        preds = probabilities.argmax(dim=1)
        correct = preds[data.test_mask].eq(data.y[data.test_mask]).sum().item()
        accuracy = correct / data.test_mask.sum().item()

        accuracies.append(accuracy)

        # 计算ROC-AUC
        y_true = data.y[data.test_mask].cpu().numpy()  # 将数据移回CPU来计算ROC-AUC
        y_score = probabilities[data.test_mask, 1].cpu().numpy()
        roc_auc = roc_auc_score(y_true, y_score)
        roc_aucs.append(roc_auc)

    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Accuracy: {accuracy}, ROC-AUC: {roc_auc}')


  0%|          | 1/1000 [00:00<15:46,  1.06it/s]

Epoch 1, Loss: 0.9256611466407776, Accuracy: 0.7303225806451613, ROC-AUC: 0.516865950395607


 10%|█         | 101/1000 [01:08<09:59,  1.50it/s]

Epoch 101, Loss: 0.4129549264907837, Accuracy: 0.8356989247311828, ROC-AUC: 0.6959294860408345


 20%|██        | 201/1000 [02:16<08:57,  1.49it/s]

...

Epoch 401, Loss: 0.40716081857681274, Accuracy: 0.8365591397849462, ROC-AUC: 0.7049740152964641


 50%|█████     | 501/1000 [05:38<05:36,  1.48it/s]

Epoch 501, Loss: 0.40728962421417236, Accuracy: 0.8374193548387097, ROC-AUC: 0.7046498408944424


 52%|█████▏    | 518/1000 [05:50<05:26,  1.48it/s]


KeyboardInterrupt: ignored

In [ ]:
NUM_FEATURES = 171
NUM_CLASSES = 2
NUM_EPOCHS = 1000

# 将模型移动到指定的设备（GPU或CPU）
# model = DeepGCN(num_features=NUM_FEATURES, num_classes=NUM_CLASSES).to(device)

# 确保数据也在同一设备上
data = data.to(device)

# 修改优化器设置
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay = 0.01)

# 存储性能指标
accuracies = []
roc_aucs = []

from sklearn.metrics import roc_auc_score
from tqdm import trange
for epoch in trange(NUM_EPOCHS):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    # 测试模型
    model.eval()
    with torch.no_grad():
        logits = model(data)
        probabilities = F.softmax(logits, dim=1)
        preds = probabilities.argmax(dim=1)
        correct = preds[data.test_mask].eq(data.y[data.test_mask]).sum().item()
        accuracy = correct / data.test_mask.sum().item()

        accuracies.append(accuracy)

        # 计算ROC-AUC
        y_true = data.y[data.test_mask].cpu().numpy()  # 将数据移回CPU来计算ROC-AUC
        y_score = probabilities[data.test_mask, 1].cpu().numpy()
        roc_auc = roc_auc_score(y_true, y_score)
        roc_aucs.append(roc_auc)

    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}, Accuracy: {accuracy}, ROC-AUC: {roc_auc}')


  0%|          | 1/1000 [00:00<12:09,  1.37it/s]

Epoch 1, Loss: 0.4069521725177765, Accuracy: 0.8296774193548387, ROC-AUC: 0.7100910225894366


 10%|█         | 101/1000 [01:08<10:01,  1.49it/s]

Epoch 101, Loss: 0.4067787528038025, Accuracy: 0.8361290322580646, ROC-AUC: 0.7066005968361593


 20%|██        | 201/1000 [02:16<08:57,  1.49it/s]

...

 80%|████████  | 801/1000 [09:01<02:13,  1.49it/s]

Epoch 801, Loss: 0.40706178545951843, Accuracy: 0.8361290322580646, ROC-AUC: 0.709020802988241


 90%|█████████ | 901/1000 [10:08<01:06,  1.48it/s]

Epoch 901, Loss: 0.40707552433013916, Accuracy: 0.8361290322580646, ROC-AUC: 0.7093278488015063


100%|██████████| 1000/1000 [11:15<00:00,  1.48it/s]


In [ ]:
print(f'Epoch {epoch+1}, Loss: {loss.item()}, Accuracy: {accuracy}, ROC-AUC: {roc_auc}')

Epoch 1000, Loss: 0.4070665240287781, Accuracy: 0.8361290322580646, ROC-AUC: 0.709537832611818


In [ ]:
model_gnn, y_score_gnn = model, y_score

## LSTM

In [ ]:
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3, dropout_rate=0.8):
        super(LSTMModel, self).__init__()
        # 定义三层LSTM，每层之间添加dropout
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout_rate)
        # Dropout层
        self.dropout = nn.Dropout(dropout_rate)
        # 定义线性层
        self.fc_token = nn.Linear(hidden_size, output_size)  # 用于 token 改变量的预测
        self.fc_label = nn.Linear(hidden_size, 1)  # 用于每个时间步的标签预测

    def forward(self, x):
        # LSTM 层
        lstm_out, _ = self.lstm(x)
        # 应用dropout
        lstm_out = self.dropout(lstm_out)
        # 在每个时间步预测 token 的改变量
        token_change = self.fc_token(lstm_out)
        # 在每个时间步预测标签
        labels = self.fc_label(lstm_out)
        return token_change, labels

# 创建模型实例
model = LSTMModel(input_size=171, hidden_size=64, output_size=171)
if torch.cuda.is_available():
    model = model.cuda()


In [ ]:
from sklearn.metrics import roc_auc_score
import torch
import numpy as np

def evaluate_model(model, eval_dataset):
    model.eval()  # 设置模型为评估模式

    # 确保模型在 GPU 上
    if torch.cuda.is_available():
        model = model.cuda()

    predictions = []
    true_labels = []

    with torch.no_grad():  # 在评估过程中不计算梯度
        for features, label in eval_dataset:
            # 将数据移至 GPU
            features = features.cuda() if torch.cuda.is_available() else features
            label = label.cuda() if torch.cuda.is_available() else label

            # 前向传播
            _, label_pred = model(features.unsqueeze(0))

            # 将预测转换为二元概率（sigmoid 函数）
            label_prob = torch.sigmoid(label_pred).squeeze().cpu().numpy()[-1]
            predictions.append(label_prob)

            # 保存真实标签
            true_labels.append(label.cpu().numpy())

    # 计算 ROC-AUC 分数
    roc_auc = roc_auc_score(true_labels, predictions)
    return roc_auc

# 使用
# valid_dataset 应该是一个适当的 DataLoader
roc_auc_score = evaluate_model(model, valid_dataset)
print(f"ROC-AUC Score: {roc_auc_score}")


ROC-AUC Score: 0.747585503377504


In [ ]:
import torch
import torch.optim as optim
from sklearn.metrics import roc_auc_score

def train_model(model, train_dataset, valid_dataset, num_epochs):
    # 检查 CUDA 是否可用，并将模型移至 GPU
    if torch.cuda.is_available():
        model = model.cuda()

    # 定义优化器和损失函数
    criterion_token = nn.MSELoss()
    criterion_label = nn.BCEWithLogitsLoss(reduction='none')
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)

    best_roc_auc = 0
    best_model = None

    for epoch in range(num_epochs):
        model.train()  # 设置模型为训练模式
        total_loss = 0
        from tqdm import tqdm
        for features, label in tqdm(train_dataset):
            features = features.cuda() if torch.cuda.is_available() else features
            label = label.cuda() if torch.cuda.is_available() else label

            optimizer.zero_grad()
            token_change_pred, label_pred = model(features.unsqueeze(0))

            actual_token_change = features[1:, :] - features[:-1, :]
            loss_token = criterion_token(token_change_pred[:, :-1], actual_token_change.unsqueeze(0))
            label = label.float().unsqueeze(0).unsqueeze(-1).expand_as(label_pred)
            loss_label = criterion_label(label_pred, label).mean()
            loss = loss_label * 10

            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_dataset)}")

        # 验证步骤
        current_roc_auc = evaluate_model(model, valid_dataset)
        print(f"Validation ROC-AUC for Epoch {epoch+1}: {current_roc_auc}")

        # 检查是否需要更新最佳模型
        if current_roc_auc > best_roc_auc:
            best_roc_auc = current_roc_auc
            best_model = model.state_dict()

    # 保存最佳模型
    if best_model is not None:
        torch.save(best_model, 'best_model.pth')
        print("Best model saved.")

    return best_model

# 使用
num_epochs = 10
best_model = train_model(model, train_dataset, valid_dataset, num_epochs)


100%|██████████| 9271/9271 [00:32<00:00, 281.43it/s]


Epoch 1/10, Loss: 4.014509346726295
Validation ROC-AUC for Epoch 1: 0.723106846867879


100%|██████████| 9271/9271 [00:32<00:00, 283.69it/s]


Epoch 2/10, Loss: 3.988595562393438
Validation ROC-AUC for Epoch 2: 0.7203586599607184


100%|██████████| 9271/9271 [00:32<00:00, 286.46it/s]


...

Epoch 8/10, Loss: 3.6213906680072747
Validation ROC-AUC for Epoch 8: 0.7065783931099935


100%|██████████| 9271/9271 [00:32<00:00, 281.95it/s]


Epoch 9/10, Loss: 3.5868179209457223
Validation ROC-AUC for Epoch 9: 0.6997320327447866


100%|██████████| 9271/9271 [00:32<00:00, 283.44it/s]


Epoch 10/10, Loss: 3.544289237001402
Validation ROC-AUC for Epoch 10: 0.6977514603707896
Best model saved.


In [ ]:
from sklearn.metrics import roc_auc_score
import torch
import numpy as np

def evaluate_model_new(model, eval_dataset):
    model.eval()  # 设置模型为评估模式

    # 确保模型在 GPU 上
    if torch.cuda.is_available():
        model = model.cuda()

    predictions = []
    true_labels = []

    with torch.no_grad():  # 在评估过程中不计算梯度
        for features, label in eval_dataset:
            # 将数据移至 GPU
            features = features.cuda() if torch.cuda.is_available() else features
            label = label.cuda() if torch.cuda.is_available() else label

            # 前向传播
            _, label_pred = model(features.unsqueeze(0))

            # 将预测转换为二元概率（sigmoid 函数）
            label_prob = torch.sigmoid(label_pred).squeeze().cpu().numpy()[-1]
            predictions.append(label_prob)

            # 保存真实标签
            true_labels.append(label.cpu().numpy())

    # 计算 ROC-AUC 分数
    predictions = (predictions+y_score_gnn)/2
    roc_auc = roc_auc_score(true_labels, predictions)
    return roc_auc

# 使用
# valid_dataset 应该是一个适当的 DataLoader
roc_auc_score_new = evaluate_model_new(model, valid_dataset)
print(f"ROC-AUC Score: {roc_auc_score_new}")


ROC-AUC Score: 0.7344713483117555


In [ ]:
y_score_gnn[:10]

array([0.15961881, 0.03937966, 0.2569339 , 0.40570876, 0.03302091,
       0.19394471, 0.7560153 , 0.2331564 , 0.6652355 , 0.18563902],
      dtype=float32)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

# Setting the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Transform class
class Transform(nn.Module):
    def __init__(self, outfea, d):
        super(Transform, self).__init__()
        self.qff = nn.Linear(outfea, outfea)
        self.kff = nn.Linear(outfea, outfea)
        self.vff = nn.Linear(outfea, outfea)

        self.ln = nn.LayerNorm(outfea)
        self.lnff = nn.LayerNorm(outfea)

        self.ff = nn.Sequential(
            nn.Linear(outfea, outfea),
            nn.ReLU(),
            nn.Linear(outfea, outfea)
        )

        self.d = d

    def forward(self, x):
        query = self.qff(x)
        key = self.kff(x)
        value = self.vff(x)

        query = torch.cat(torch.split(query, self.d, -1), 0).permute(0,2,1,3)
        key = torch.cat(torch.split(key, self.d, -1), 0).permute(0,2,3,1)
        value = torch.cat(torch.split(value, self.d, -1), 0).permute(0,2,1,3)

        A = torch.matmul(query, key)
        A /= (self.d ** 0.5)
        A = torch.softmax(A, -1)

        value = torch.matmul(A ,value)
        value = torch.cat(torch.split(value, x.shape[0], 0), -1).permute(0,2,1,3)
        value += x

        value = self.ln(value)
        x = self.ff(value) + value
        return self.lnff(x)

# PositionalEncoding class
class PositionalEncoding(nn.Module):
    def __init__(self, outfea):
        super(PositionalEncoding, self).__init__()
        self.outfea = outfea if outfea % 2 == 0 else outfea + 1

    def forward(self, x):
        max_len = x.size(1)
        pe = torch.zeros(max_len, self.outfea).to(x.device)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, self.outfea, 2).float() * -(math.log(10000.0) / self.outfea))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).unsqueeze(2)  # [1, T, 1, F]
        x = x + pe[:,:,:,:x.size(-1)]  # Adjust positional encoding size to match input size
        return x

# SGNN class
class SGNN(nn.Module):
    def __init__(self, outfea):
        super(SGNN, self).__init__()
        self.ff = nn.Sequential(
            nn.Linear(outfea, outfea),
            nn.Linear(outfea, outfea)
        )
        self.ff1 = nn.Linear(outfea, outfea)

    def forward(self, x):
        p = self.ff(x)
        a = torch.matmul(p, p.transpose(-1,-2))
        R = torch.relu(torch.softmax(a, -1)) + torch.eye(x.shape[1]).to(device)

        D = (R.sum(-1) ** -0.5)
        D[torch.isinf(D)] = 0.
        D = torch.diag_embed(D)

        A = torch.matmul(torch.matmul(D, R), D)
        x = torch.relu(self.ff1(torch.matmul(A, x)))
        return x

# GRU class
class GRU(nn.Module):
    def __init__(self, outfea):
        super(GRU, self).__init__()
        self.ff = nn.Linear(2*outfea, 2*outfea)
        self.zff = nn.Linear(2*outfea, outfea)
        self.outfea = outfea

    def forward(self, x, xh):
        r, u = torch.split(torch.sigmoid(self.ff(torch.cat([x, xh], -1))), self.outfea, -1)
        z = torch.tanh(self.zff(torch.cat([x, r*xh], -1)))
        x = u * z + (1-u) * xh
        return x


class STGNNwithGRU(nn.Module):
    def __init__(self, outfea):
        super(STGNNwithGRU, self).__init__()
        self.sgnn = SGNN(outfea)  # 使用单个SGNN实例
        self.gru = GRU(outfea)

    def forward(self, x):
        B, T, N, F = x.shape
        hidden_state = torch.zeros([B, N, F]).to(device)
        output = []

        for i in range(T):
            gx = self.sgnn(x[:, i, :, :])  # 在每个时间步使用相同的SGNN实例
            gh = hidden_state
            if i != 0:
                gh = self.sgnn(hidden_state)  # 重复使用SGNN实例
            hidden_state = self.gru(gx, gh)
            output.append(hidden_state)

        output = torch.stack(output, 1)
        return output


# Continuing with the STGNNwithGRU class
class STGNNwithGRU(nn.Module):
    def __init__(self, outfea):
        super(STGNNwithGRU, self).__init__()
        self.sgnn = SGNN(outfea)  # Using a single SGNN instance
        self.gru = GRU(outfea)

    def forward(self, x):
        B, T, N, F = x.shape
        hidden_state = torch.zeros([B, N, F]).to(device)
        output = []

        for i in range(T):
            gx = self.sgnn(x[:, i, :, :])  # Use the same SGNN instance at each time step
            gh = hidden_state
            if i != 0:
                gh = self.sgnn(hidden_state)  # Reuse the SGNN instance
            hidden_state = self.gru(gx, gh)
            output.append(hidden_state)

        output = torch.stack(output, 1)
        return output

# STGNN class
class STGNN(nn.Module):
    def __init__(self, infea, outfea, L, d):
        super(STGNN, self).__init__()
        self.start_emb = nn.Linear(infea, outfea)
        self.adjust_emb = nn.Linear(outfea, infea)  # Additional layer for dimension adjustment
        self.end_emb = nn.Linear(infea, infea)

        self.stgnnwithgru = nn.ModuleList([STGNNwithGRU(outfea) for i in range(L)])
        self.positional_encoding = PositionalEncoding(outfea)
        self.transform = nn.ModuleList([Transform(outfea, d) for i in range(L)])

        self.fc_token_change = nn.Linear(infea, infea)  # Predicts next time step change
        self.fc_label = nn.Linear(infea, 1)  # Predicts label

        self.L = L

    def forward(self, x):
        B, T, F = x.shape
        x = x.unsqueeze(2)  # Add spatial dimension [B, T, 1, F]
        x = self.start_emb(x)
        for i in range(self.L):
            x = self.stgnnwithgru[i](x)
        x = self.positional_encoding(x)
        for i in range(self.L):
            x = self.transform[i](x)
        x = self.adjust_emb(x)  # Adjust dimensions
        x = self.end_emb(x)
        x = x.squeeze(2)  # Remove spatial dimension [B, T, F]

        token_change = self.fc_token_change(x)
        labels = self.fc_label(x).squeeze(-1)

        return token_change, labels

# Instantiate and test the model
model = STGNN(infea=171, outfea=64, L=3, d=8).to(device)
test_input = torch.rand(1, 13, 171).to(device)
token_change, labels = model(test_input)

# Check the shapes of the outputs
token_change.shape, labels.shape

(torch.Size([1, 13, 171]), torch.Size([1, 13]))

In [ ]:
num_epochs = 10
best_model = train_model(model, train_dataset, valid_dataset, num_epochs)

KeyboardInterrupt: ignored

#Fusion all pipeline

In [ ]:
import torch

def extract_features_and_labels(dataset, feature_indices=None, retain_TS=False):
    features_list = []
    labels_list = []

    for features, label in dataset:
        # 如果提供了特征索引，只选择这些特征
        if feature_indices is not None:
            selected_features = features[:, feature_indices]
        else:
            selected_features = features

        # 检查是否保留时间序列
        if retain_TS:
            # 保留时间序列的第一个维度
            if selected_features.shape[0] < 30:
                # 如果少于30天，使用最后一天的值进行填充
                padding = torch.ones((30 - selected_features.shape[0], *selected_features.shape[1:])) * selected_features[-1]
                selected_features = torch.cat([selected_features, padding], dim=0)
            else:
                # 只取最后30天的值
                selected_features = selected_features[-30:]
        else:
            # 对时间维度进行平均
            selected_features = selected_features.mean(dim=0)  # 对时间维度取平均值

            # 添加住院天数作为一个新的特征
            num_days = features.shape[0]  # 时间维度的大小就是住院天数
            selected_features = torch.cat([selected_features, torch.tensor([num_days], dtype=torch.float32)])

        features_list.append(selected_features)  # 存储特征
        labels_list.append(label)  # 存储标签

    # 将列表转换为Tensor
    features_tensor = torch.stack(features_list)
    labels_tensor = torch.stack(labels_list)

    return features_tensor, labels_tensor

In [ ]:
#@title indice
demo_indices = list(range(3))  # demo 特征在前3个位置

icd_cols_indices = list(range(3, 3+91))  # icd_cols 特征在接下来的91个位置


lab_cols_indices = list(range(3+91, 3+91+36))  # lab_cols 特征在接下来的36个位置
#加一个time feature

med_cols_indices = list(range(3+91+36, 3+91+36+41))  # med_cols 特征在接下来的41个位置
#加一个feature为这些value的standarlized sum（？）

# 打印结果
print("demo_indices:", demo_indices)
print("icd_cols_indices:", icd_cols_indices)
print("lab_cols_indices:", lab_cols_indices)
print("med_cols_indices:", med_cols_indices)

demo_indices: [0, 1, 2]
icd_cols_indices: [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93]
lab_cols_indices: [94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129]
med_cols_indices: [130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170]


In [ ]:
#@title 三个model
import torch.nn as nn

class SimpleNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=3, dropout_rate=0.8):
        super(LSTMModel, self).__init__()
        # 定义三层LSTM，每层之间添加dropout
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers=num_layers,
                            batch_first=True, dropout=dropout_rate)
        # Dropout层
        self.dropout = nn.Dropout(dropout_rate)
        # 定义线性层
        self.fc_token = nn.Linear(hidden_size, output_size)  # 用于 token 改变量的预测
        self.fc_label = nn.Linear(hidden_size, 3)  # 用于每个时间步的标签预测

    def forward(self, x):
        # LSTM 层
        lstm_out, _ = self.lstm(x)
        # 应用dropout
        lstm_out = self.dropout(lstm_out)
        # 在每个时间步预测 token 的改变量
        token_change = self.fc_token(lstm_out)
        # 在每个时间步预测标签
        labels = self.fc_label(lstm_out)
        return token_change, labels


class LogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)
        #self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc(x)
        #out = self.sigmoid(out)
        return out

class SimpleFusionNetwork(nn.Module):
    def __init__(self, input_dim, hidden_dim, dropout_rate=0.5):
        super(SimpleFusionNetwork, self).__init__()

        # Input layer
        self.input_layer = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

        # Output layer
        self.output_layer = nn.Sequential(
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid() #因为pred是prob所以框在0-1
        )

    def forward(self, x):
        x = self.input_layer(x)
        x = self.output_layer(x)
        return x

class VerySimpleFusionNetwork(nn.Module):
    def __init__(self, input_dim, output_dim):
      super(VerySimpleFusionNetwork, self).__init__()
      self.fc = nn.Linear(input_dim, output_dim)
      self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        out = self.fc(x)
        out = self.sigmoid(out)
        return out


class FusionModel(nn.Module):
    def __init__(self, lstm_input_size, lstm_hidden_size, lstm_output_size, nn_input_dim, nn_output_dim, fusion_input_dim, fusion_hidden_dim, dropout_rate=0.5):
        super(FusionModel, self).__init__()

        # 定义LSTM模型
        self.lstm_model = LSTMModel(lstm_input_size, lstm_hidden_size, lstm_output_size, dropout_rate=dropout_rate)

        # 定义逻辑回归模型
        self.nn_model = LogisticRegression(nn_input_dim, nn_output_dim)

        # 定义融合网络
        self.fusion_model = SimpleFusionNetwork(fusion_input_dim, fusion_hidden_dim, dropout_rate=dropout_rate)

    def forward(self, batch_features_1, batch_features_2):
        _, lstm_out = self.lstm_model(batch_features_2)
        nn_out = self.nn_model(batch_features_1)

        # 取 LSTM 输出的最后一个时间步
        lstm_out_last = lstm_out[:, -1, :]

        # 融合 LSTM 和 NN 的输出
        fusion_in = torch.cat((nn_out, lstm_out_last), dim=1)
        fusion_out = self.fusion_model(fusion_in)

        return fusion_out

In [ ]:
#initialze model + dataloader
nn_model = LogisticRegression(input_dim=136,output_dim=1)
lstm_model = LSTMModel(input_size=36, hidden_size=128, output_size=50)
fusion_model = VerySimpleFusionNetwork(input_dim=2, output_dim=1)

from torch.utils.data import TensorDataset, DataLoader

train_features_1 = torch.tensor(train_features_1, dtype=torch.float32)
train_features_2 = torch.tensor(train_features_2, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.float32)

test_features_1 = torch.tensor(test_features_1, dtype=torch.float32)
test_features_2 = torch.tensor(test_features_2, dtype=torch.float32)
test_labels = torch.tensor(test_labels, dtype=torch.float32)


train_dataset = TensorDataset(train_features_1, train_features_2, train_labels)
test_dataset = TensorDataset(test_features_1, test_features_2, test_labels)

#dataloader
batch_size = 1000
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

<ipython-input-79-5e9fe672406f>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_features_1 = torch.tensor(train_features_1, dtype=torch.float32)
<ipython-input-79-5e9fe672406f>:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_features_2 = torch.tensor(train_features_2, dtype=torch.float32)
<ipython-input-79-5e9fe672406f>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  train_labels = torch.tensor(train_labels, dtype=torch.float32)
<ipython-input-79-5e9fe672406f>:12: UserWarning: To copy construct from a tensor, it is recomme

In [ ]:
demo_indices = list(range(3))  # demo 特征在前3个位置

icd_cols_indices = list(range(3, 3+91))  # icd_cols 特征在接下来的91个位置


lab_cols_indices = list(range(3+91, 3+91+36))  # lab_cols 特征在接下来的36个位置
#加一个time feature

med_cols_indices = list(range(3+91+36, 3+91+36+41))


print(
    data['feat_dict']['11674366_29673314'][1][3:3+91]
)
print(data['feat_dict']['11674366_29673314'][1][3:3+91])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0])

In [ ]:
#train and evaluate: part 1
from sklearn.metrics import roc_auc_score
import torch.optim as optim

input_dim = 172  # 这是一个示例值，你需要根据你的数据来设置
hidden_dim = 10  # 这也是一个示例值，你可以根据你的需求来设置
output_dim = 1  # 这是一个例子，假设你的问题是二分类问题

nn_model = LogisticRegression(input_dim, output_dim)

num_epochs=10
learning_rate=1e-4

criterion = torch.nn.BCELoss()
optimizer = optim.SGD(list(nn_model.parameters()), lr=learning_rate)  # 只优化 nn_model 的参数

for epoch in range(num_epochs):
    for batch_features_1, _, batch_labels in test_dataloader:  # 只使用 feature1 和 labels

        nn_out = nn_model(batch_features_1)

        # 计算loss
        batch_labels = batch_labels.unsqueeze(1)
        loss = criterion(nn_out, batch_labels)  # 直接使用 nn_out 计算 loss

        # 反向传播和优化
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(f'Loss: {loss.item()}')

    #一个epoch结束
    nn_model.eval()

    preds = []
    true_labels = []

    #infer+evaluate
    with torch.no_grad():
        for batch_features_1, _,batch_labels in test_dataloader:  # 只使用 feature1 和 labels
            nn_out = nn_model(batch_features_1)

            probas = nn_out.squeeze().cpu().numpy()
            preds.extend(probas)
            true_labels.extend(batch_labels.squeeze().cpu().numpy())

    # 计算本次epoch的AUC
    auc = roc_auc_score(true_labels, preds)
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}, Test AUC: {auc}')

    #训练模式
    nn_model.train()

In [ ]:
# 实例化模型
input_dim = 100  # 例如，你的输入维度是100
output_dim = 1  # 例如，你的输出维度是1
model = LogisticRegression(input_dim, output_dim)

# 定义损失函数和优化器
criterion = nn.BCEWithLogitsLoss()  # 由于你没有在模型中使用sigmoid，所以这里使用BCEWithLogitsLoss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)  # 使用SGD优化器

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    for batch_features, batch_labels in train_dataloader:
        # 前向传播
        outputs = model(batch_features)
        loss = criterion(outputs, batch_labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # 打印损失信息
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

ValueError: ignored

In [ ]:
batch_labels

tensor([0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 1., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 1., 1., 0., 0., 0., 0.])